In [2]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, SGDRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd
import numpy as np

df = pd.read_csv('토마토_전지역.csv', encoding = 'cp949')
df.drop(['True'], axis = 1, inplace = True)

In [3]:
df = df[df['지역'] == '서울']

In [4]:
y = df['가격']
x = df[['최고기온(°C)', '연', '월', '일', '연속강수', '생산자물가', '가조시간(hr)']]

In [84]:
df.corr()['생산자물가']



C:\Users\gjaischool1\AppData\Local\Temp\ipykernel_13744\4221932800.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()['생산자물가']


평균기온(°C)         -0.340508
최저기온(°C)         -0.342849
최고기온(°C)         -0.333537
강수 계속시간(hr)      -0.074923
10분 최다 강수량(mm)   -0.119778
1시간 최다강수량(mm)    -0.118022
일강수량(mm)         -0.073073
최대 순간 풍속(m/s)    -0.015327
최대 풍속(m/s)        0.010191
평균 풍속(m/s)        0.083755
평균 이슬점온도(°C)     -0.305224
최소 상대습도(%)       -0.085200
평균 상대습도(%)       -0.080567
평균 증기압(hPa)      -0.342931
평균 현지기압(hPa)      0.396409
최고 해면기압(hPa)      0.414358
최저 해면기압(hPa)      0.373499
평균 해면기압(hPa)      0.397016
가조시간(hr)         -0.503746
합계 일조시간(hr)      -0.019289
일 최심신적설(cm)       0.014264
일 최심적설(cm)        0.016358
합계 3시간 신적설(cm)    0.015169
평균 지면온도(°C)      -0.383380
최저 초상온도(°C)      -0.356313
9-9강수(mm)        -0.075717
연                 0.313080
월                 0.061760
일                -0.000800
연속강수             -0.125644
가격                0.821926
생산자물가             1.000000
소비자물가             0.897977
Name: 생산자물가, dtype: float64

In [51]:
#______________________________________________________________________________________________________________________________
from statsmodels.stats.outliers_influence import variance_inflation_factor

def convert_from_scientific(x):
    return '{:.6f}'.format(x)

# apply 함수를 사용하여 Facter 컬럼의 값을 변환

d_df = df[['연', '월', '일', '가조시간(hr)', '최고기온(°C)', '생산자물가']]
# d_df
vif = pd.DataFrame()
vif['Facter'] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
vif['features'] = x.columns
vif = vif.sort_values('Facter').reset_index(drop = True)
vif

vif['Facter'] = vif['Facter'].apply(convert_from_scientific)
vif

,Facter,features
0,1.264544,연속강수
1,4.192968,일
2,4.741453,월
3,4.770624,최고기온(°C)
4,17.742292,생산자물가
5,29.339225,연


In [52]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2721 entries, 31 to 43770
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   최고기온(°C)  2721 non-null   float64
 1   연         2721 non-null   int64  
 2   월         2721 non-null   int64  
 3   일         2721 non-null   int64  
 4   연속강수      2721 non-null   int64  
 5   생산자물가     2721 non-null   float64
dtypes: float64(2), int64(4)
memory usage: 148.8 KB


In [6]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, SGDRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd
import numpy as np

tr_x, tt_x, tr_y, tt_y = train_test_split(x, y, random_state = 3, test_size = 0.2)
tr_x.shape, tt_x.shape, tr_y.shape, tt_y.shape

mms = MinMaxScaler().fit(tr_x)
mms_train_x = mms.transform(tr_x)
mms_test_x = mms.transform(tt_x)


# # 1번
q = PolynomialFeatures(degree = 10)
x_q = q.fit_transform(mms_train_x)
x_tt_q = q.transform(mms_test_x)

# m = Ridge()
# m.fit(x_q, tr_y)
# m.score(x_q, tr_y), m.score(x_tt_q, tt_y)

## 2번
m = SGDRegressor(max_iter=200, shuffle=False, verbose=1)#, eta0=1e-4)
m.fit(x_q, tr_y)
print(cross_val_score(m, x_q, tr_y).mean())
m.score(x_q, tr_y), m.score(x_tt_q, tt_y)


## 3번
# m = DecisionTreeRegressor()
# m.fit(mms_train_x, tr_y)

# print(cross_val_score(m, mms_train_x, tr_y).mean())
# m.score(mms_train_x, tr_y), m.score(mms_test_x, tt_y)


# # 4번
# m = LinearRegression()
# m.fit(mms_train_x, tr_y)

# print(cross_val_score(m, mms_train_x, tr_y).mean())
# m.score(mms_train_x, tr_y), m.score(mms_test_x, tt_y)

-- Epoch 1
Norm: 2365.64, NNZs: 888030, Bias: 1301.866328, T: 2176, Avg. loss: 2832044.242623
Total training time: 91.87 seconds.
-- Epoch 2
Norm: 2281.97, NNZs: 888030, Bias: 1415.327775, T: 4352, Avg. loss: 360569.386993
Total training time: 173.81 seconds.
-- Epoch 3
Norm: 2344.16, NNZs: 888030, Bias: 1505.214049, T: 6528, Avg. loss: 225978.988101
Total training time: 257.56 seconds.
-- Epoch 4
Norm: 2401.45, NNZs: 888030, Bias: 1565.985457, T: 8704, Avg. loss: 193244.866407
Total training time: 334.78 seconds.
-- Epoch 5
Norm: 2449.25, NNZs: 888030, Bias: 1609.665001, T: 10880, Avg. loss: 177860.648949
Total training time: 413.36 seconds.
-- Epoch 6
Norm: 2489.74, NNZs: 888030, Bias: 1642.778797, T: 13056, Avg. loss: 168737.428373
Total training time: 496.68 seconds.
-- Epoch 7
Norm: 2524.93, NNZs: 888030, Bias: 1668.917794, T: 15232, Avg. loss: 162550.909753
Total training time: 580.63 seconds.
-- Epoch 8
Norm: 2556.22, NNZs: 888030, Bias: 1690.201279, T: 17408, Avg. loss: 157963.

KeyboardInterrupt: 